In [24]:
import boto3
import json
from botocore.exceptions import ClientError

In [2]:
client = boto3.client('events')

In [3]:
event_bus_name = 'Orders'
source = 'com.aws.orders'
detail_type = 'Order Notification'
detail = '{"category":"lab-supplies","value":415,"location":"eu-west"}'

In [4]:
# Create a eventbridge bus
def create_eventbridge_bus(bus_name):
    """
    Creates an EventBridge bus with the given name.
    
    Args:
        bus_name (str): The name of the EventBridge bus to create.
        
    Returns:
        dict: The response from EventBridge containing bus details.
    """
    client = boto3.client('events')
    
    try:
        response = client.create_event_bus(
            Name=bus_name
        )
        return response
    except client.exceptions.ResourceAlreadyExistsException:
        print(f"EventBridge bus '{bus_name}' already exists.")
        return None

In [5]:
create_eventbridge_bus(bus_name=event_bus_name)

{'EventBusArn': 'arn:aws:events:us-east-1:637423169504:event-bus/Orders',
 'ResponseMetadata': {'RequestId': '46b9dc2b-89b0-4c39-9296-8f0b4ce2736d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '46b9dc2b-89b0-4c39-9296-8f0b4ce2736d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '72',
   'date': 'Mon, 08 Jul 2024 16:27:31 GMT'},
  'RetryAttempts': 0}}

In [10]:
# Create an EventBridge Rule
rule_event = {
  "source": ["com.aws.orders"],
  "detail": {
    "category": [{
      "prefix": "lab-supplies"
    }]
  }
}

In [6]:
detail = {
  "category": "lab-supplies-12",
  "value": 1500,
  "location": "eu-west"
}

In [7]:
try:
    response = client.put_events(
        Entries=[
          {
            'Source': source,
            'DetailType': detail_type,
            'Detail': json.dumps(detail),
            'EventBusName': event_bus_name
          }
        ]
    )
    print('Event sent to the event bus ' + event_bus_name)
    print('EventID is ' + response['Entries'][0]['EventId'])
except Exception as e:
    print(e)

Event sent to the event bus Orders
EventID is e51d5be0-3e37-1ae5-e554-6b37dde79461


### Calculating EventBridge PutEvents event entry size
You can send custom events to EventBridge by using the PutEvents action. You can batch multiple event entries into one request for efficiency. The total entry size must be less than 256KB. You can calculate the entry size before you send the events.

In [14]:
def get_size_kb(entry):  # Assuming 'entry' is a dictionary-like object with the necessary keys
    size = 0

    if entry.get("time"):  # Check if 'Time' key exists and has a value
        size += 14  # Size of the timestamp representation (adjust if needed)

    size += len(entry.get("source", "").encode("utf-8"))  # Length of 'Source' in bytes
    size += len(entry.get("detail-type", "").encode("utf-8"))  # Length of 'DetailType' in bytes

    if entry.get("detail"):
        detail_data = json.dumps(entry["detail"]).encode("utf-8")
        size += len(detail_data)  # Length of 'Detail' in bytes

    if entry.get("resources"):
        for resource in entry["resources"]:
            if resource:
                resource_data = json.dumps(resource).encode("utf-8")
                size += len(resource_data) # Length of each resource in bytes
        
    return round(size / 1024, 2) # Return size in kilobytes


In [72]:
with open("users_1k.json", "r") as f:
    event_data = f.read()
    json_object = json.loads(event_data)

In [73]:
len(json_object)

898

In [84]:
event_sample = {
    "version": "0",
    "id": "ddbec620-9261-eed9-f7fd-bb4a6024c333",
    "detail-type": "Order Notification",
    "source": "com.aws.orders",
    "account": "637423169504",
    "time": "2024-07-07T16:10:19Z",
    "region": "us-east-1",
    "resources": [],
    "detail": json_object[:750]
}

In [85]:
size_kb = get_size_kb(event_sample)
print(f"The size of the event sample is: {size_kb} KB")
print(f"Size of the event must be less than 256KB")

The size of the event sample is: 248.92 KB
Size of the event must be less than 256KB


In [86]:
detail = {
  "category": "lab-supplies-12",
  "value": 1500,
  "location": "eu-west",
  "users": json_object[:750]
}

In [87]:
try:
    response = client.put_events(
        Entries=[
          {
            'Source': source,
            'DetailType': detail_type,
            'Detail': json.dumps(detail),
            'EventBusName': event_bus_name
          }
        ]
    )
    print('Event sent to the event bus ' + event_bus_name)
    print('EventID is ' + response['Entries'][0]['EventId'])
except ClientError as e:
    print('Error Message: {}'.format(e.response['Error']['Message']))
    raise e
except Exception as e:
    raise e

Event sent to the event bus Orders
EventID is b9c60c8e-3a13-cd80-b3fe-2c81076c5932
